# Studi Kasus Pertemuan ke-13: Dimensionality Reduction
Dalam studi kasus ini, anda akan menggunakan PCA pada dataset MNIST.     
Tujuan penggunaan PCA adalah untuk mengurangi jumlah fitur dari dataset MNIST.      
   
Dua pertanyaan yang muncul adalah   
1. Apakah PCA selalu dapat meningkatkan kecepatan dari pelatihan model?
2. Bagaimana akurasi atau _performance_ model pada dataset yang sudah dikurangi fiturnya? Masih OK? Atau lebih jelek? Atau malahan meningkat?

Untuk menjawab dua pertanyaan tersebut, anda akan melakukan dua eksperimen, yaitu:
1. Membandingkan _performance_ _random forest_ pada dataset MNIST sebelum dan sesudah dikenakan proses PCA.
2. Membandingkan _performance_ _random forest_ pada dataset MNIST sebelum dan sesudah dikenakan proses PCA.

Kita import-import library yang dibutuhkan.

In [9]:
from packaging import version
import sklearn
assert version.parse(sklearn.__version__)\
 >= version.parse("1.0.1")

In [2]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

Mari kita load dataset MNIST.

In [4]:
mnist = fetch_openml('mnist_784', as_frame=False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
mnist.data.shape

(70000, 784)

`mnist.data` terdiri dari 70.000 (tujuh puluh ribu) _instances_.    
Mari kita bagi dataset menjadi _train set_ sebanyak 60.000 (enam puluh ribu) dan _test set_ sebanyak 10.000 (sepuluh ribu).


In [6]:
X_train = mnist.data[:60000]
y_train = mnist.target[:60000]

X_test = mnist.data[60000:]
y_test = mnist.target[60000:]

Mari kita buat `RandomForestClassifier` dengan `n_estimators=100` dan `random_state=42`.

In [7]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

Kita hitung juga waktu training yang dibutuhkan.

In [10]:
%time rnd_clf.fit(X_train, y_train)

CPU times: user 54.4 s, sys: 252 ms, total: 54.7 s
Wall time: 58.1 s


RandomForestClassifier(random_state=42)

Selanjutnya, kita gunakan model yang sudah dilatih untuk memprediksi test set.

In [11]:
y_pred = rnd_clf.predict(X_test)

Kita gunakan `accuracy_score` untuk menghitung akurasi dari model.

In [12]:
accuracy_score(y_test, y_pred)

0.9705

## _Dimensionality Reduction_ pada MNIST dan Random Forest

Selanjutnya, kita gunakan PCA to reduce the dataset's dimensionality dengan _explained variance ratio_ sebesar 95%.

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

Selanjutnya, kita latih _Random Forest classifier_ pada dataset yang sudah dikurangi dimensinya.      
Kita hitung juga berapa lama proses training tersebut.   
   
**Catatan**: Waktu yang dibutuhkan dicatat oleh **Wall time**-nya   .

In [14]:
rnd_clf_with_pca = RandomForestClassifier\
(n_estimators=100, random_state=42)

%time rnd_clf_with_pca.fit(X_train_reduced, y_train)

CPU times: user 2min 15s, sys: 276 ms, total: 2min 15s
Wall time: 2min 25s


RandomForestClassifier(random_state=42)

### Pertanyaan refleksi
- Bagaimana perbandingan waktu _training_ _random forest_ pada dataset lengkap dan waktu _training_ _random forest_ pada dataset yang sudah direduksi? Perbandingan waktu dataset lengkap lebih cepat dibanding dataset yang sudah direduksi

Selanjutnya, mari kita evaluasi _random forest_ pada _test set_

In [15]:
X_test_reduced = pca.transform(X_test)

Kita hitung prediksi model pada _test set_.

In [16]:
y_pred = rnd_clf_with_pca.predict(X_test_reduced)

Mari kita hitung akurasi (_accuracy_) model.

In [17]:
accuracy_score(y_test, y_pred)

0.9481

### Pertanyaan refleksi
- Bagaimana perbandingan akurasi _random forest_ pada dataset lengkap dan akurasi pada dataset yang sudah direduksi? akurasi pada dataset lengkap lebih besar dibanding dataset yang sudah direduksi
- Ketika PCA digunakan, apakah akurasi _random forest_ **menurun jauh** dibandingkan random forest ketika PCA tidak digunakan? menurun cukup kecil sih dari awalnya **0.9705** menjadi **0.9481**

## _Dimensionality Reduction_ pada Dataset dan _Stochastic Gradient Descent_ (SGD)

Sekarang kita akan ganti model, dari **random forest** menjadi **stochastic gradient descent** (SGD).   
Buatlah SGD dengan `random_state=42`

In [18]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
%time sgd_clf.fit(X_train, y_train)

CPU times: user 3min 12s, sys: 396 ms, total: 3min 12s
Wall time: 3min 17s


SGDClassifier(random_state=42)

Selanjutnya, model SGD yang sudah dilatih digunakan untuk memprediksi `X_test`.

In [19]:
y_pred = sgd_clf.predict(X_test)

Mari kita hitung akurasi (_accuracy_) model.

In [20]:
accuracy_score(y_test, y_pred)

0.874

### Pertanyaan refleksi
- Bagaimanakah perbandingan waktu _training_ `SGDClassifier` dengan waktu _training_ `RandomForestClassifier` pada dataset lengkap? untuk perbandingan,
`RandomForestClassifier` masih jauh lebih cepat

Saatnya, kita coba melatih model SGD (dengan `random_state=42`) tapi kali ini dengan menggunakan dataset yang sudah direduksi.

In [21]:
sgd_clf_with_pca =  SGDClassifier(random_state=42)
%time sgd_clf_with_pca.fit(X_train_reduced, y_train)

CPU times: user 46.4 s, sys: 81.4 ms, total: 46.5 s
Wall time: 46.8 s


SGDClassifier(random_state=42)

### Pertanyaan refleksi
- Bagaimana perbandingan waktu _training_ SGD pada dataset lengkap dan waktu _training_ SGD pada dataset yang sudah direduksi? wah untuk waktu training
saat dataset sudah direduksti jauh  lebih cepat dari yang tidak direduksi

Selanjutnya, model SGD dengan PCA yang sudah dilatih digunakan untuk memprediksi `X_test`.

In [22]:
y_pred = sgd_clf_with_pca.predict(X_test_reduced)

Mari kita hitung akurasi (_accuracy_) model.

In [23]:
accuracy_score(y_test, y_pred)

0.8959

### Pertanyaan refleksi
- Bagaimana perbandingan akurasi SGD pada dataset lengkap dan akurasi SGD pada dataset yang sudah direduksi? wah akurasi SGD pada dataset yang sudah direduksi
cukup meningkat dari 0.874 menjadi 0.8959
- Ketika PCA digunakan, apakah akurasi SGD **menurun jauh** dibandingkan ketika PCA tidak digunakan? tidak bahkan malah meningkat

### Pertanyaan refleksi
Coba anda pikirkan kembali dua pertanyaan yang ditanyakan di awal, yaitu:  
1. Apakah PCA selalu dapat meningkatkan kecepatan dari pelatihan model? Bisa meningkatkan kecepatan pelatihan model tetapi juga tidak untuk beberapa classifier.
2. Bagaimana akurasi atau _performance_ model pada dataset yang sudah dikurangi fiturnya? Masih OK? Atau lebih jelek? Atau malahan meningkat? Akurasinya menjadi meningkat walaupun tidak terlalu besar.


<center>
    <h1>The End</h1>
</center>    